In [2]:
library(caret)
library(mlbench)
library(GA)
library(doParallel)

Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'mlbench' was built under R version 3.5.3"Loading required package: foreach
Loading required package: iterators
Package 'GA' version 3.1.1
Type 'citation("GA")' for citing this R package in publications.
Warning message:
"package 'doParallel' was built under R version 3.5.3"Loading required package: parallel


In [28]:
p <- "E:\\1.DATA_ANALYTICS\\17-Analise Preditiva Avancada\\Exercicio_6\\GA_Exercise.csv"
df <- read.csv(file= p, header=TRUE, sep=";")

In [29]:
head(df)

AGE,WORKCLASS,FNLWGT,EDUCATION,EDU_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,COUNTRY,INCOME
<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,B
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,B
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,B
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,B
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,B
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,B


In [30]:
set.seed(314)
options(warn=-1)

# Particionamentos dos Dados
trainIndex <- createDataPartition(df$INCOME,p=.7,list=FALSE)

trainData <- df[trainIndex,]
testData <- df[-trainIndex,]

trainX <-trainData[,-ncol(trainData)]
testX <- testData[,-ncol(trainData)]
y <- trainData$INCOME

In [31]:
# Modelo Sem Feature Selection
set.seed(314)
cv <- trainControl(method = "repeatedcv", number = 10, 
                   savePredictions = TRUE, classProbs=TRUE)

model_5.0 <- train(INCOME~. , data = trainData, method = "C5.0Tree",
                   trControl = cv, 
                   na.action=na.exclude)


pred_5.0 <- predict(model_5.0 ,newdata=testData)
confusionMatrix(data=pred_5.0, testData$INCOME)

Confusion Matrix and Statistics

          Reference
Prediction   A   B
         A  21  12
         B  28 153
                                         
               Accuracy : 0.8131         
                 95% CI : (0.7543, 0.863)
    No Information Rate : 0.771          
    P-Value [Acc > NIR] : 0.08105        
                                         
                  Kappa : 0.402          
 Mcnemar's Test P-Value : 0.01771        
                                         
            Sensitivity : 0.42857        
            Specificity : 0.92727        
         Pos Pred Value : 0.63636        
         Neg Pred Value : 0.84530        
             Prevalence : 0.22897        
         Detection Rate : 0.09813        
   Detection Prevalence : 0.15421        
      Balanced Accuracy : 0.67792        
                                         
       'Positive' Class : A              
                                         

In [32]:
registerDoParallel(4) 
getDoParWorkers() 

set.seed(314)
ctrl <- gafsControl(functions = caretGA,
                    genParallel=TRUE,
                    allowParallel=TRUE,
                    method = "cv")

                     
obj <- gafs(x = trainX,
            y = y,
            iters = 2,
            popSize = 2,
            gafsControl = ctrl,
            method = "C5.0Tree")

obj

[1] 4


Genetic Algorithm Feature Selection

504 samples
14 predictors
2 classes: 'A', 'B' 

Maximum generations: 2 
Population per generation: 2 
Crossover probability: 0.8 
Mutation probability: 0.1 
Elitism: 0 

Internal performance values: Accuracy, Kappa
Subset selection driven to maximize internal Accuracy 

External performance values: Accuracy, Kappa
Best iteration chose by maximizing external Accuracy 
External resampling method: Cross-Validated (10 fold) 

During resampling:
  * the top 5 selected variables (out of a possible 14):
    EDUCATION (90%), RELATIONSHIP (90%), OCCUPATION (80%), AGE (70%), EDU_NUM (70%)
  * on average, 9.4 variables were selected (min = 2, max = 14)

In the final search using the entire training set:
   * 14 features selected at iteration 2 including:
     AGE, WORKCLASS, FNLWGT, EDUCATION, EDU_NUM ... 
   * external performance at this iteration is

   Accuracy       Kappa 
     0.8196      0.4328 


In [39]:
final <- obj$ga$final
final

pred_final <- predict(obj$fit,newdata=testData)
confusionMatrix(data=pred_final, testData$INCOME)

[1] "AGE"            "WORKCLASS"      "FNLWGT"         "EDUCATION"     
 [5] "EDU_NUM"        "MARITAL_STATUS" "OCCUPATION"     "RELATIONSHIP"  
 [9] "RACE"           "SEX"            "CAPITAL_GAIN"   "CAPITAL_LOSS"  
[13] "HOURS_PER_WEEK" "COUNTRY"

Confusion Matrix and Statistics

          Reference
Prediction   A   B
         A  27  15
         B  22 150
                                          
               Accuracy : 0.8271          
                 95% CI : (0.7696, 0.8752)
    No Information Rate : 0.771           
    P-Value [Acc > NIR] : 0.02785         
                                          
                  Kappa : 0.4844          
 Mcnemar's Test P-Value : 0.32394         
                                          
            Sensitivity : 0.5510          
            Specificity : 0.9091          
         Pos Pred Value : 0.6429          
         Neg Pred Value : 0.8721          
             Prevalence : 0.2290          
         Detection Rate : 0.1262          
   Detection Prevalence : 0.1963          
      Balanced Accuracy : 0.7301          
                                          
       'Positive' Class : A               
                                          

In [34]:
# Modelo Sem Feature Selection
set.seed(314)
cv <- trainControl(method = "repeatedcv", number = 10, 
                   savePredictions = TRUE, classProbs=TRUE)

model_5.0 <- train(INCOME~AGE+WORKCLASS+FNLWGT+EDUCATION+EDU_NUM+MARITAL_STATUS+OCCUPATION+RELATIONSHIP+RACE+SEX+CAPITAL_GAIN+CAPITAL_LOSS+HOURS_PER_WEEK+COUNTRY, data = trainData, method = "C5.0Tree",
                   trControl = cv, 
                   na.action=na.exclude)


pred_5.0 <- predict(model_5.0 ,newdata=testData)
confusionMatrix(data=pred_5.0, testData$INCOME)

Confusion Matrix and Statistics

          Reference
Prediction   A   B
         A  21  12
         B  28 153
                                         
               Accuracy : 0.8131         
                 95% CI : (0.7543, 0.863)
    No Information Rate : 0.771          
    P-Value [Acc > NIR] : 0.08105        
                                         
                  Kappa : 0.402          
 Mcnemar's Test P-Value : 0.01771        
                                         
            Sensitivity : 0.42857        
            Specificity : 0.92727        
         Pos Pred Value : 0.63636        
         Neg Pred Value : 0.84530        
             Prevalence : 0.22897        
         Detection Rate : 0.09813        
   Detection Prevalence : 0.15421        
      Balanced Accuracy : 0.67792        
                                         
       'Positive' Class : A              
                                         

In [35]:
# Modelo Sem Feature Selection
set.seed(314)
cv <- trainControl(method = "repeatedcv", number = 10, 
                   savePredictions = TRUE, classProbs=TRUE)

model_5.0 <- train(INCOME~EDUCATION+CAPITAL_GAIN, data = trainData, method = "C5.0Tree",
                   trControl = cv, 
                   na.action=na.exclude)


pred_5.0 <- predict(model_5.0 ,newdata=testData)
confusionMatrix(data=pred_5.0, testData$INCOME)

Confusion Matrix and Statistics

          Reference
Prediction   A   B
         A  10   1
         B  39 164
                                         
               Accuracy : 0.8131         
                 95% CI : (0.7543, 0.863)
    No Information Rate : 0.771          
    P-Value [Acc > NIR] : 0.08105        
                                         
                  Kappa : 0.2722         
 Mcnemar's Test P-Value : 4.909e-09      
                                         
            Sensitivity : 0.20408        
            Specificity : 0.99394        
         Pos Pred Value : 0.90909        
         Neg Pred Value : 0.80788        
             Prevalence : 0.22897        
         Detection Rate : 0.04673        
   Detection Prevalence : 0.05140        
      Balanced Accuracy : 0.59901        
                                         
       'Positive' Class : A              
                                         

In [4]:
desafio <- "E:\\1.DATA_ANALYTICS\\17-Analise Preditiva Avancada\\Exercicio_1\\BostonRealEstate_Exercise.csv"
df_2 <- read.csv(file= desafio, header=TRUE, sep=",")
head(df_2)

NBATH,CRIM,NROOMS,TAX,DISC,AGE,PRICE
<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>
3,0.81,5,70.19,111.03,70,111.03
1,0.68,3,75.10,123.09,17,118.94
1,0.69,2,78.89,223.98,3,167.12
3,0.89,5,96.97,68.86,97,68.86
1,0.73,3,39.79,161.36,15,145.42
1,0.56,2,40.68,137.97,17,127.69


In [5]:
# Divisão Treino & Teste
set.seed(987)
trainIndex <- createDataPartition(df_2$PRICE, p = .7, list = FALSE)

# Bases de Treino & Teste
set.seed(987)
dfTrain <- df_2[ trainIndex,]
dfTest  <- df_2[-trainIndex,]

# Vetores Genéticos
set.seed(987)
vector_1 <- c('CRIM','NROOMS','AGE','PRICE')
vector_2 <- c('TAX','DISC','AGE','PRICE')

# Bases de Treino para cada Modelo
dfTrain_1 <- df_2[,vector_1]
dfTrain_2 <- df_2[,vector_2]

# Modelos
model_1 <- train(PRICE~., data = dfTrain_1 ,method = "lm")
model_2 <- train(PRICE~., data = dfTrain_2 ,method = "lm")

In [6]:
model_1

Linear Regression 

834 samples
  3 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 834, 834, 834, 834, 834, 834, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  40.14682  0.3395054  30.96571

Tuning parameter 'intercept' was held constant at a value of TRUE

In [7]:
model_2

Linear Regression 

834 samples
  3 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 834, 834, 834, 834, 834, 834, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  23.25566  0.7777663  17.41788

Tuning parameter 'intercept' was held constant at a value of TRUE